In [7]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=6

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],0:2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# unit sphere constraint
u=3
p=floor(Int64,n/u)

I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
f=sum(generate_random_poly(I[j]) for j in 1:p)


g=-(x.^2).+1/u



m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/7)

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)


randx=2*rand(n).-1
randx=randx./sqrt(u)

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make constraints feasible
    end
end

l=length(h)
println("Number of equality constraints: l=",l)
println("====================")

k=2

println("Relaxed order: k=",k)
println("====================")

***Problem setting***
Number of variable: n=6
Number of inequality constraints: m=6
Number of equality constraints: l=1
Relaxed order: k=2


In [8]:
using ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=true);

In [9]:
using ctpPOP

opt_val1=ctpPOP.POP_CS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
    maxit=Int64(1e10),tol=1e-2,use_eqcons_to_get_constant_trace=false,check_tol_each_iter=true)

------------------------------------------------------
The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------
  Number of cliques: p=2
  Largest clique size: u=4
  Largest constant trace: a=4.111111111111111
  Number of blocks: omega=9
  Number of equality consraints: zeta=181
  Size of the largest block: s^max=15
Modeling time:
  0.012576 seconds (25.84 k allocations: 2.348 MiB)
iter=1   val=-9.160738790638973   gap=0.011663840776587522   feas=7.662391083258684
iter=2   val=-3.557371602434557   gap=0.573835698607101   feas=2.0691204134736574
iter=4   val=-2.1565949543564193   gap=0.5468445671254273   feas=1.6627140583622182
iter=8   val=-2.135783577077469   gap=0.2643929320856459   feas=1.0482022584336537
iter=16   val=-2.435513133865892   gap=0.1556979611528819   feas=0.42966632752631906
iter=32   val=-2.2080608465804734   gap=0.11524391017121466   feas=0.1812746111892033
iter=64   val=-2.341756561019352   gap=0.02893360992547715   feas=0.0

-2.3809935239629847

In [10]:
using ctpPOP

opt_val1=ctpPOP.POP_CS_LMBM(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
    tol=1e-2,use_eqcons_to_get_constant_trace=false)

------------------------------------------------------
The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------
  Number of cliques: p=2
  Largest clique size: u=4
  Largest constant trace: a=4.111111111111111
  Number of blocks: omega=9
  Number of equality consraints: zeta=181
  Size of the largest block: s^max=15
Modeling time:
  0.010657 seconds (25.84 k allocations: 2.347 MiB)
**LMBM solver:
---------------
| Parameters: |
---------------
n:       181
maxtime: 300000.000000
na:      2
mcu:     5
mc:      7
rpar: 
ipar: 
 Entry to LMBM:
 NIT=    1  NFE=    1  F= 0.90431119D+00  WK= 0.1675D-01  QK= 0.8374D-02
 NIT=    2  NFE=    2  F= 0.89430129D+00  WK= 0.1675D-01  QK= 0.8374D-02
 NIT=    3  NFE=    3  F= 0.88611380D+00  WK= 0.1675D-01  QK= 0.8374D-02
 NIT=    4  NFE=    4  F= 0.88042802D+00  WK= 0.1675D-01  QK= 0.8374D-02
 NIT=    5  NFE=    5  F= 0.87817233D+00  WK= 0.1675D-01  QK= 0.8374D-02
 NIT=    6  NFE=   10  F= 0.87817101D+00  WK= 

-8.913526219316687

In [13]:
using TSSOS


@time opt,sol,data=cs_tssos_first(Vector{SparseMatrixCSC{UInt8,UInt32}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,CS="MD",TS=false);

------------------------------------------------------
The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 100             
  Cones                  : 0               
  Scalar variables       : 16              
  Matrix variables       : 8               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 

In [12]:
#using ctpPOP

#optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)